**INSTALLS**

In [3]:
# # 1a. Upgrade pip & wheel
# !pip install --upgrade pip setuptools wheel

# # 1b. Pre-install a binary wheel for blis so pip won’t try to compile it
# !pip install blis==0.7.9

# # 1c. Install spaCy itself
# !pip install spacy

# !pip install langdetect


**IMPORTS**

In [4]:
import re
import pandas as pd
import spacy
import torch
import torch.nn.functional as F
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from langdetect import detect

from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

**Loading & Preprocessing**

In [5]:
# Load the uploaded dataset
file_path = "AWARE_Comprehensive.csv"
df = pd.read_csv(file_path)
df.shape

(11321, 14)

In [6]:
df

,domain,app,review_id,sentence_id,title,review,sentence,rating,is_opinion,category,term,from,to,sentiment
0,productivity,things-3,c9274c0a-a120-4e09-816b-7a8ba3a16634,00808934-e8b9-42fa-b37f-cfeac234bbdd,Difficult to update from Things 2,This new version of Things has an entirely dif...,This new version of Things has an entirely dif...,3,True,usability,new version,6.0,17.0,positive
1,productivity,notability,e633e20a-07c1-4a5e-80b1-b104b6cf6a61,00a8d4a4-9c8e-4d1c-9085-ffd1f62ae039,Great app,I have been using this app for over 3 years no...,All those contractors were blown away by how e...,5,False,NaN,NaN,NaN,NaN,NaN
2,productivity,microsoft-word,506230e3-cc98-4233-be40-89e52d53990c,011cd77b-ebbd-4589-af12-9792975b02b9,Same word. Same problem,A lot of people use word because its very stan...,"I tell them what happens, and they say “so is ...",1,True,general,NaN,NaN,NaN,negative
3,productivity,things-3,69d44a5e-218f-4f55-8a99-6cca55d43ca1,014a7d01-f6c0-408a-897b-f6b36cdd8543,Incredible Planner for Students,I was originally skeptical on paying $7 on a t...,"The ease of use, simplicity, and great functio...",5,True,effectiveness,functionality,40.0,53.0,positive
4,productivity,notability,3d0b634c-d402-47e8-ba7d-bf6209fed826,01539a8d-ebde-4cfe-9b5b-3fa10f49fa3e,Recording and syncing,This used to be my go to app for note taking. ...,I’m disappointed that even after a year they h...,1,True,cost,money,95.0,100.0,negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11316,games,among-us-,70d02d85-60b0-4c94-a43d-0307f089e36e,166c4b99-efe7-4d4c-8afa-3e09f4937796,My experience with Among Us,I used to play Among Us but not really anymore...,Third is the community.,3,False,NaN,NaN,NaN,NaN,NaN
11317,games,subway-surfers,5431fed8-7c83-4ea1-bc4e-c2282c6f1bbd,42d9aa0e-7183-4148-996f-8e90e7852abf,ARE YOU KIDDING ME!!!????,I have been playing this game since the releas...,I want this Salma girl SO MUCH.,2,False,NaN,NaN,NaN,NaN,NaN
11318,games,monopoly,5880f046-3a22-4dd9-8b01-ab6dea2bd4f6,2a82db36-9f4a-49f4-8d70-551a900222a9,Freezes up consistently!!!,NEW PROBLEM: the game consistently mis moves t...,Ticks you off especially after spending an hou...,2,False,NaN,NaN,NaN,NaN,NaN
11319,games,among-us-,8765dd75-e764-4d94-a2ea-f23c8894908d,92ab173f-cad1-4fa9-bcc3-eb1a2dfed83b,GREAT GAME! But.....,So this is a great game! and if your not playi...,If your wondering the community is ok but cert...,5,False,NaN,NaN,NaN,NaN,NaN


In [7]:

# 1. Drop non-English reviews before any cleaning
def is_english(text: str) -> bool:
    try:
        return detect(text) == 'en'
    except:
        return False

# Apply language filter on the raw content column
df = df[df['sentence'].apply(is_english)].reset_index(drop=True)


In [8]:
def clean_text(text):
  text = text.lower()
  text = re.sub(r'<[^>]+>', ' ', text)                    # remove HTML
  text = re.sub(r'http\S+|www\.\S+', ' ', text)       # remove URLs
  text = re.sub(r'[^a-z0-9\s]', ' ', text)              # keep alphanumeric
  text = re.sub(r'\s+', ' ', text).strip()              # collapse spaces
  return text

# Apply cleaning
df['sentence'] = df['sentence'].apply(clean_text)



In [9]:
# DROP DUBLICATES
# 1. Load a pretrained sentence‐embedding model
#    (you may need to install via: pip install sentence-transformers)
embedder = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
sentences = df['sentence'].tolist()
embeddings = embedder.encode(sentences, convert_to_numpy=True)
sim_matrix = cosine_similarity(embeddings, embeddings)

In [11]:
# 4. Mark semantically duplicate rows for removal
threshold = 0.90  # adjust as needed; 0.90 means very high semantic overlap
n = len(sentences)
to_drop = set()

for i in range(n):
    if i in to_drop:
        continue
    # For j > i, check similarity
    for j in range(i + 1, n):
        if j in to_drop:
            continue
        if sim_matrix[i, j] >= threshold:
            # Mark the later index for dropping
            to_drop.add(j)

# 5. Drop semantically duplicate rows from df
df = df.drop(df.index[list(to_drop)]).reset_index(drop=True)

# 6. (Optional) Verify how many were removed
print(f"Dropped {len(to_drop)} semantically duplicate reviews, "
      f"remaining {len(df)} reviews.")


Dropped 1108 semantically duplicate reviews, remaining 10067 reviews.


In [12]:
df.shape

(10067, 14)

In [13]:
def extract_candidate_aspects(doc):
    """
    Return a list of normalized aspect phrases from a single review Doc.
    Applies:
      - pronoun/short‐chunk filtering
      - stopword‐only filtering
      - normalize_aspect checks (strip articles, drop contractions, enforce alphanumeric, etc.)
    """
    aspects = set()
    for chunk in doc.noun_chunks:
        text = chunk.text.strip().lower()

        # Skip pronouns or overly short chunks
        if chunk.root.pos_ == "PRON" or len(text) <= 2:
            continue
        # Skip chunks composed entirely of stopwords
        if all(token.is_stop for token in chunk):
            continue
        aspects.add(text)
    return list(aspects)


**ABSA MODEL**

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

absa_model_name = "yangheng/deberta-v3-large-absa-v1.1"
tokenizer_absa  = AutoTokenizer.from_pretrained(absa_model_name)
model_absa      = AutoModelForSequenceClassification.from_pretrained(absa_model_name)
model_absa.to(device)
model_absa.eval()

tokenizer_config.json:   0%|          | 0.00/397 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 1024, padding_idx=0)
      (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-23): 24 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (key_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (value_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (pos_dropout): Dropout(p=0.1, inplace=False)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNo

In [15]:
# -----------------------------
# 4. Batched ABSA Predictor
# -----------------------------
def predict_batch(review: str, aspects: list):
    texts   = [review] * len(aspects)
    encoding = tokenizer_absa(
        text=texts,
        text_pair=aspects,
        truncation=True,
        padding="longest",
        max_length=128,
        return_tensors="pt"
    )
    # Move all tensors to GPU/CPU
    encoding = {k: v.to(device) for k, v in encoding.items()}

    with torch.no_grad():
        logits = model_absa(**encoding).logits       # shape: [len(aspects), 3]
    probs = F.softmax(logits, dim=1)                 # shape: [len(aspects), 3]
    labels = torch.argmax(probs, dim=1)              # shape: [len(aspects)]

    label_map = {0: "negative", 1: "neutral", 2: "positive"}
    results = []
    for i, lbl_id in enumerate(labels.tolist()):
        lbl = label_map[int(lbl_id)]
        conf = float(probs[i, lbl_id].item())
        results.append((lbl, conf))
    return results

**USAGE**

In [ ]:

# YOU CAN ADJUST HERE
df = df.iloc[:1000].reset_index(drop=True)

In [17]:

# Load spaCy model
nlp = spacy.load("en_core_web_sm")  # CPU version

rows = []
for idx, row in tqdm(df.iterrows(), total=len(df)):
    review_id   = row.get("review_id", idx)
    star_rating = row["rating"]        # 1–5 star rating column
    text        = row["sentence"]

    # 5a. Parse review with spaCy to get a Doc
    doc = nlp(text)

    # 5b. Extract aspects for this review
    candidate_aspects = extract_candidate_aspects(doc)
    if not candidate_aspects:
        continue

    # ────────────────────────────────
    snippet = text[:80] + "…" if len(text) > 80 else text
    print(f"\nReview ID: {review_id}  (Stars: {star_rating})")
    print(f"  \"{snippet}\"")
    print("  Detected Aspects & Polarity:")

    # 5c. Run ABSA in one batch for all extracted aspects
    predictions = predict_batch(text, candidate_aspects)
    for asp, (label, conf) in zip(candidate_aspects, predictions):
        rows.append({
            "review_id": review_id,
            "aspect": asp,
            "predicted_polarity": label,
            "confidence": conf,
            "star_rating": star_rating
        })
        print(f"    • {asp:20s} → {label:8s} (conf={conf:.2f})")

absa_results = pd.DataFrame(rows)


  0%|          | 1/1000 [00:00<03:03,  5.45it/s]


Review ID: 11cc7692-fd2e-4839-a1f3-8cf5116743f8  (Stars: 3)
  "discords recent update had changed and displayed a call display on the lock scre…"
  Detected Aspects & Polarity:
    • an iphone call       → neutral  (conf=1.00)
    • recent update        → negative (conf=0.80)
    • the lock screen      → neutral  (conf=1.00)
    • a call display       → neutral  (conf=0.99)

Review ID: 8a3cbb81-c603-4f72-9c9c-0476fc36a8e5  (Stars: 2)
  "how do i undo the purchase of 1 99 i just did on accident"
  Detected Aspects & Polarity:


  0%|          | 3/1000 [00:00<02:47,  5.96it/s]

    • the purchase         → negative (conf=0.98)
    • accident             → negative (conf=0.74)

Review ID: f45fc850-9b2f-447c-b9e4-445a1e645da8  (Stars: 1)
  "sending me death threats and gruesome cartel murders trying to extort money from…"
  Detected Aspects & Polarity:
    • money                → neutral  (conf=0.97)
    • gruesome cartel murders → neutral  (conf=0.97)
    • death threats        → neutral  (conf=0.95)


  0%|          | 4/1000 [00:00<02:37,  6.34it/s]

...


Review ID: 365124c7-1419-4b82-9189-c8a60fd8e2f4  (Stars: 3)
  "they can counter back twice more each and then they either reject or take the tr…"
  Detected Aspects & Polarity:
    • the trade            → neutral  (conf=0.56)
    • their turn           → negative (conf=0.57)

Review ID: 8d31d8c1-df41-428f-8287-0efa4435c31f  (Stars: 3)
  "by making all these improvements i think you might just get me to give you five …"
  Detected Aspects & Polarity:
    • five stars           → neutral  (conf=0.71)
    • all these improvements → positive (conf=0.87)

Review ID: 5431fed8-7c83-4ea1-bc4e-c2282c6f1bbd  (Stars: 2)
  "we can t get these cakes in five days especially with all this lag"
  Detected Aspects & Polarity:
    • these cakes          → positive (conf=0.53)
    • five days            → negative (conf=1.00)
    • all this lag         → negative (conf=0.99)

Review ID: b7392acd-d7ca-4c32-bdc6-104938a4a667  (Stars: 5)
  "apple are you listening"
  Detected Aspects & Polarity:


100%|█████████▉| 996/1000 [01:00<00:00, 14.34it/s]

    • apple                → neutral  (conf=0.98)

Review ID: 5aef1f08-23ae-4777-a2ab-229522a185bb  (Stars: 1)
  "i use the fuji scansnap to scan all paperwork and have gone completely paperless…"
  Detected Aspects & Polarity:
    • my office            → neutral  (conf=1.00)
    • the fuji scansnap    → neutral  (conf=0.94)

Review ID: 81c89cca-ccfb-4e0f-9e5f-6d5514ac12e0  (Stars: 1)
  "i have tried to connect to my guild chat sever and it always says out of date"
  Detected Aspects & Polarity:
    • my guild             → negative (conf=1.00)
    • date                 → negative (conf=0.97)

Review ID: 076c8d13-214b-43b2-b067-1923f7f42fe6  (Stars: 3)
  "the only reason i put 3 stars is because the spend v reward is the biggest joke …"
  Detected Aspects & Polarity:
    • any game             → negative (conf=0.80)
    • the biggest joke     → neutral  (conf=0.52)
    • the only reason      → negative (conf=0.83)
    • 3 stars              → negative (conf=0.89)
    • the spend v re

100%|██████████| 1000/1000 [01:00<00:00, 16.40it/s]

    • boom                 → neutral  (conf=0.84)
    • a few buttons        → neutral  (conf=0.80)

Review ID: 0bca0bba-df15-4394-9014-47e7cc7af1cd  (Stars: 3)
  "and please patch the glitch"
  Detected Aspects & Polarity:
    • the glitch           → negative (conf=1.00)

Review ID: 8f342472-7487-42bc-810a-a2dcb057331f  (Stars: 1)
  "this is the last time mobile will ask"
  Detected Aspects & Polarity:
    • mobile               → neutral  (conf=0.94)
    • the last time        → neutral  (conf=0.96)


In [18]:

# from google.colab import files
# absa_results.to_csv('absa_results.csv', index=False)
# files.download('absa_results.csv')

In [19]:
# Load the ABSA results dataset
# absa_results = pd.read_csv('absa_results.csv')